    임베딩 레이어에 들어가기 위해서 정수 인코딩이 필요함.

##### 임베딩 레이어
    임베딩 층은 다음과 같은 세 개의 인자를 받습니다.

    vocab_size = 텍스트 데이터의 전체 단어 집합의 크기입니다.
    output_dim = 워드 임베딩 후의 임베딩 벡터의 차원입니다.
    input_length = 입력 시퀀스의 길이입니다. 만약 갖고있는 각 샘플의 길이가 500개이라면 이 값은 500입니다.

In [ ]:
vocab_size = 20000
output_dim = 128
input_length = 500

v = Embedding(vocab_size, output_dim, input_length=input_length)

##### 임베딩 층 사용하기

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

In [2]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 고려하여 +1
print('단어 집합 :',vocab_size)

단어 집합 : 16


In [3]:
X_encoded = tokenizer.texts_to_sequences(sentences)
print('정수 인코딩 결과 :',X_encoded)

정수 인코딩 결과 : [[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


In [4]:
max_len = max(len(l) for l in X_encoded)
print('최대 길이 :',max_len)

최대 길이 : 4


In [5]:
X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train = np.array(y_train)
print('패딩 결과 :')
print(X_train)

패딩 결과 :
[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [6]:
## 테스트할 간단한 모델 생성

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

embedding_dim = 4

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 1s - loss: 0.7034 - acc: 0.2857
Epoch 2/100
1/1 - 0s - loss: 0.7015 - acc: 0.4286
Epoch 3/100
1/1 - 0s - loss: 0.6996 - acc: 0.4286
Epoch 4/100
1/1 - 0s - loss: 0.6977 - acc: 0.5714
Epoch 5/100
1/1 - 0s - loss: 0.6958 - acc: 0.5714
Epoch 6/100
1/1 - 0s - loss: 0.6939 - acc: 0.5714
Epoch 7/100
1/1 - 0s - loss: 0.6920 - acc: 0.5714
Epoch 8/100
1/1 - 0s - loss: 0.6901 - acc: 0.5714
Epoch 9/100
1/1 - 0s - loss: 0.6882 - acc: 0.7143
Epoch 10/100
1/1 - 0s - loss: 0.6863 - acc: 0.7143
Epoch 11/100
1/1 - 0s - loss: 0.6844 - acc: 0.7143
Epoch 12/100
1/1 - 0s - loss: 0.6825 - acc: 0.7143
Epoch 13/100
1/1 - 0s - loss: 0.6807 - acc: 0.7143
Epoch 14/100
1/1 - 0s - loss: 0.6788 - acc: 0.8571
Epoch 15/100
1/1 - 0s - loss: 0.6769 - acc: 0.8571
Epoch 16/100
1/1 - 0s - loss: 0.6750 - acc: 1.0000
Epoch 17/100
1/1 - 0s - loss: 0.6731 - acc: 1.0000
Epoch 18/100
1/1 - 0s - loss: 0.6712 - acc: 1.0000
Epoch 19/100
1/1 - 0s - loss: 0.6693 - acc: 1.0000
Epoch 20/100
1/1 - 0s - loss: 0.6674 - a

##### 사전 훈련된 워드 임베딩(Pre-Trained Word Embedding) 사용하기
    케라스의 Embedding()을 사용하여 처음부터 임베딩 벡터값을 학습하기도 하지만, 때로는 이미 훈련되어져
    있는 워드 임베딩을 가져와서 이를 임베딩 벡터로 사용하기도 합니다. 훈련 데이터가 적은 상황이라면 케라스의
    Embedding()으로 해당 문제를 풀기에 최적화 된 임베딩 벡터값을 얻는 것이 쉽지 않습니다. 이 경우 해당
    문제에 특화된 것은 아니지만 보다 많은 훈련 데이터로 이미 Word2Vec이나 GloVe 등으로 학습되어져 있는
    임베딩 벡터들을 사용하는 것이 성능의 개선을 가져올 수 있습니다.

    사전 훈련된 GloVe와 Word2Vec 임베딩을 사용해서 모델을 훈련시키는 실습을 진행해봅시다.

    GloVe 다운로드 링크 : http://nlp.stanford.edu/data/glove.6B.zip
    Word2Vec 다운로드 링크 : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM

    훈련 데이터는 앞서 사용했던 데이터에 동일한 전처리까지 진행된 상태라고 가정하겠습니다.

In [7]:
print(X_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]
